In [1]:
#step 1: install/upgrade the latest genai SDK
%pip install google-genai --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 2.1 MB/s eta 0:00:00


In [2]:
#import the genai library
from google import genai

In [3]:
#step2: AIStudio: read the api key from the user data
from google.colab import userdata
client = genai.Client(api_key=userdata.get("GEMINI_KEY"))

#If you want to read from environment keys
#import os
#client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [4]:
#step2: Vertex AI: read the api key from the user data

#for vertex, you need to have a service account or login/auth
#login as the current user for colab testing purposes
from google.colab import auth
auth.authenticate_user()

#then configure the client with the project and location settings

#Uncomment the following if you are running on Vertex setup

#using the colab userdata
#from google.colab import userdata
#client = genai.Client(vertexai=True, #ensure to turn on VertexAI
#                      project=userdata.get("GOOGLE_CLOUD_PROJECT"),
#                      location=userdata.get("GOOGLE_CLOUD_REGION")) # e.g., 'us-central1'

#or use the folliwing if you want to read from environment keys
#import os
#client = genai.Client(vertexai=True,
#                      project=os.environ["GOOGLE_CLOUD_PROJECT"],
#                      location=os.environ["GOOGLE_CLOUD_REGION"]) # e.g., 'us-central1'

In [6]:
sample_file = client.files.upload(file="/content/book.txt")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.name}")


Uploaded file 'None' as: files/bparlkhtfd92


In [7]:
print(f"File state: {sample_file.state}")
while sample_file.state.name == "PROCESSING":
    print("Waiting for processing...")
    # Sleep for a few seconds before checking again
    import time
    time.sleep(10)
    # Get the latest file information
    sample_file = client.files.get_file(name=sample_file.name)
    print(f"File state: {sample_file.state}")

if sample_file.state.name == "FAILED":
    raise ValueError("File processing failed.")

print(f"File {sample_file.name} is ready for use.")

File state: FileState.ACTIVE
File files/bparlkhtfd92 is ready for use.


In [16]:
from google.genai import types
with open('/content/book.txt', 'r') as file:
    content = file.read()
# model = genai.GenerativeModel(model_name="gemini-2.5-flash-preview-04-17")

# Include the file URI in the prompt
prompt = [
    "Describe the contents of this text.",
    types.Part
    (
        text=content
    )
]

# Make the generate content request
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt
)
print(response.text)

This text is the Project Gutenberg eBook of The Odyssey, translated by Samuel Butler.

It includes:

*   **Licensing information:** Details about the copyright status of the eBook, its free usage terms, and the Project Gutenberg license.
*   **Title, Author, and Translator:** Specifies "The Odyssey" as the title, Homer as the author, and Samuel Butler as the translator.
*   **Release information:** The initial release date and last update date of the eBook.
*   **Credits:** Acknowledgments to Jim Tinsley and David Widger.
*   ***The full text of the Odyssey, rendered into English prose**:* Divided into books, with titles.
*   **Prefaces:** Two prefaces, one to the first edition, detailing Butler's theories about the authorship and setting of The Odyssey, and another to the second edition. The preface provides insights into his translation philosophy and touches on points of academic debate surrounding the text.

    



In [20]:
# Get file details
file_details = client.files.get(name=sample_file.name)
print(f"Retrieved file: {file_details.display_name}, MIME type: {file_details.mime_type}")

# List files
for file in client.files.list():
    print(file.name, file.display_name)

# Delete the file when done
client.files.delete(name=sample_file.name)
print(f"Deleted file {sample_file.name}")

Retrieved file: None, MIME type: text/plain
files/bparlkhtfd92 None
files/tawfzf80hvih None
Deleted file files/bparlkhtfd92
